# Compare iJL1678b-ME and iEco1689-ME

Code for reproducing panels of Figure 2: 2a, 2b, and 2c

## Setup environment

In [1]:
import coralme
from coralme.builder.helper_functions import *
import pickle
import pandas
import os
import tqdm
import json
import matplotlib.pyplot as plt
import numpy as np
import sympy
pandas.set_option('display.max_colwidth', None)
plt.rcParams['svg.fonttype'] = 'none'

## Functions to load and inspect models

Load a ME-model

In [2]:
def load_me(filename='me_model.pickle'):
    with open(filename, "rb") as f:
        return pickle.load(f)

## Load Genomes

In [8]:
features = {}
for org in get_org_dirs('./clean/'):
    p = './clean/{}/inputs/genome.gb'.format(org)
    contigs = get_genbank_contigs(p)
    count = 0
    for c in contigs:
        for f in c.features:
            if not f.type == 'gene': continue
            count += 1
    features[org] = count